In [16]:
import modeling.surname_classifier as cl
import modeling.surname_data as sd
import modeling.surname_common as sc
import torch.nn as nn
import torch



In [38]:
df_categories = sd.load_df_categories()
df_training, df_test = sd.load_df_surnames()

In [7]:
n_hidden = 128
n_categories = len(df_categories)
rnn = cl.RNN(sc.N_LETTERS, n_hidden, n_categories)

In [10]:
hidden =torch.zeros(1, n_hidden)

In [13]:
input = sc.letter_to_tensor('A')
output, next_hidden = rnn(input, hidden)

In [14]:
input = sc.surname_to_tensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)

In [17]:
criterion = nn.NLLLoss()

In [18]:
cl.LEARNING_RATE = 0.005 

In [19]:
def train(category_tensor, surname_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(surname_tensor.size()[0]):
        output, hidden = rnn(surname_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-cl.LEARNING_RATE, p.grad.data)

    return output, loss.item()

In [78]:
example = df_training.iloc[1]
surname_tensor = sc.surname_to_tensor(example['normalized'])
category_tensor = torch.tensor([int(df_categories[df_categories['category'] == example['category']]['index'])])
print(category_tensor)

tensor([16])


In [109]:
current_loss = 0
for iter in range(0,len(df_training)-1):
    example = df_training.iloc[iter]
    surname_tensor = sc.surname_to_tensor(example['normalized'])
    category_tensor = torch.tensor([int(df_categories[df_categories['category'] == example['category']]['index'])])
    output, loss = train(category_tensor, surname_tensor)
    current_loss += loss
    if iter % 100 == 0:
        print(loss)

2.7569572925567627
2.366332530975342
2.9685089588165283
2.3418092727661133
0.727475643157959
3.866908073425293
0.39691638946533203
0.3667154312133789
2.594465732574463
0.16710424423217773
1.5123193264007568
1.5906217098236084
1.371978521347046
0.6301910877227783
1.0447921752929688
0.5450763702392578
3.6713852882385254
1.9992893934249878
2.085850477218628
2.032996654510498
0.2861795425415039
1.3608720302581787
0.043761253356933594
0.6757445335388184
1.2281408309936523
1.4503154754638672
0.4978067874908447
3.538618803024292
1.2626335620880127
1.4289023876190186
1.1449460983276367
2.6895108222961426
1.3286616802215576
0.2486262321472168
3.4068102836608887
3.8934152126312256
0.2222909927368164
1.2577195167541504
2.372375249862671
0.10960054397583008
1.8627066612243652
0.08706426620483398
0.6778597831726074
2.687189817428589
0.9294288158416748
0.07575416564941406
0.40099430084228516
1.389535665512085
1.5651230812072754
1.289027214050293
0.14660406112670898
2.800637722015381
1.29814529418945

In [107]:
len(df_training)

15055

In [115]:
cl.predict(rnn,'Nakata')

'[[-0.35572338104248047, 10], [-1.9404892921447754, 0], [-2.8636114597320557, 14]]'

In [116]:
torch.save(rnn,'data/model/rnn.pickle')